In [1]:
import os, re
os.chdir("/Users/Pace/Documents/Education/Universität Potsdam/Semester 4 - Summer 2024/Evaluating Chatp-Optimized Language Models/Project")
from backends import ModelSpec, Model, get_model_for, load_model_registry
#need to add api key
#need to add model_name in the dict
#need to create key.json
load_model_registry()  # load default model registry from backends folder
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106") # works without registered entry when openai_api.py backend is available


In [2]:
import re

#system prompt
with open('games/ludo/resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    sys_prompt = f.read()

#initial instruction
with open('games/ludo/resources/initial_prompts/multitoken_revised_v1.txt', 'r') as f:
    content = f.read()

prefix = "MY MOVE:"


In [3]:
lmm: Model = get_model_for(THIS_MODEL)

lmm.set_gen_args(temperature=0.0, max_tokens=400)


In [4]:
prefix = "MY MOVE:"

# This works. Currently, problem with prompt potentially -> GPT fucks up counting the blocks again. 
class Game():
    def __init__(self, lmm, initial_msg, sys_prompt):
        self.num_fields = 23
        self.current_pos = {'X': 0, 'Y': 0}  # Track positions of both tokens
        self.msg = initial_msg  # initial message -> contains game instructions
        self.last_response = []
        self.sys_prompt = sys_prompt  # system prompt
        self.lmm = lmm  # language model object
        self.rolls = [6, 5, 4, 5, 6, 6, 2,3,6,5,1,2,5,3,4]  # list of rolls per turn
        self.board = "□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □"
        self.turn = 0  # turn counter
        self.context = []  # memorized context
        self.current_state = ''  # current state of the board
        self.prefix = "MY MOVE:"  # used to parse the response
        self.sign = " -> "
        self.sep = ' ; '
        self.tokens_out = {'X': False, 'Y': False}  # Track which tokens are on the board
        self.six_count = 0  # Counter for number of 6s rolled

    def make_move(self):
            if self.turn == 0:  # first turn prompt is slightly different
                in_m = f"Are you ready to play?\nBeginning State: {self.board}\n Turn num: {self.turn}, Roll: {self.rolls[self.turn]}. Where will you move your token? Let's think step by step."
                self.add_message(self.msg + '\n' + in_m)
                init_prompt, init_resp, init_resp_text = self.lmm.generate_response(self.context)
                move = self.parse_reply(init_resp_text)
                self.last_response = init_resp_text
                
                if self.check_move(move, self.rolls[self.turn]) == True:
                    print(self.board)
                    self.add_message(init_resp_text, role='assistant')
                    for token in move.keys():
                        if move[token] > 0:
                            self.tokens_out[token] = True
                        
                    self.update_board(move)                 
                    for token in move.keys():
                        self.current_pos[token] = move[token]                 

                    self.turn += 1  # updates step
                    print(self.current_state)
                else:
                    print(f'fail at turn: {self.turn}')
                    print(f'roll: {self.rolls[self.turn]}')
                    print(f'Move: {move}')
                    print(f'{init_resp_text}')
                    print(self.current_state)
            else:
                # if not first turn
                m = f"Current state: {self.current_state}\n Next turn number: {self.turn}, Roll: {self.rolls[self.turn]}.\n Where will you move your token? Let's think step by step."
                self.add_message(m, role='user')
                given_prompt, response, response_text = self.lmm.generate_response(self.context)
                move = self.parse_reply(response_text)
                self.last_response = response_text
                if self.check_move(move, self.rolls[self.turn]) == True:
                    print(self.current_state)
                    self.add_message(response_text, role='assistant')
                    for token in move.keys():
                        if move[token] > 0:
                            self.tokens_out[token] = True
                            
                    self.update_board(move)
                    for token in move.keys():
                        self.current_pos[token] = move[token]

                    self.turn += 1
                    
                    print(self.current_state)
                else:
                    print(f'fail at turn: {self.turn}')
                    print(f'roll: {self.rolls[self.turn]}')
                    print(f'move: {move}')
                    print(f'{response_text}')
                    print(self.current_state)
    
    
    
    def token_moved(self, move):
        tl = {}
        
        for token in move.keys():
            
            prev = self.current_pos[token]
            cur = move[token]
        
            if prev != cur:
                tl[token] = True
            else:
                tl[token] = False
                
        return tl
    
    def both_tokens_moved(self, move):
            l = []
            for token in move.keys():
                if self.current_pos[token] != move[token]:
                    l.append(True)
                else:
                    l.append(False)
            if all(l):
                return True
            else:
                return False
    
    def find_selected(self, tokens_moved):
        selected = None
        for t in tokens_moved.keys():
            if tokens_moved[t] == True:
                selected = t
                
        return selected
    
    def check_move(self, move, roll):
        state = []
        
        if self.both_tokens_moved(move):
            print('f0')
            return False
        
        tokens_moved = self.token_moved(move)
        selected = self.find_selected(tokens_moved)
        #print(tokens_moved)
        #print(selected)
        if not selected: # if no move has been made
            for token in move.keys(): # iterate through tokens 
                if not self.tokens_out[token]: # if token is not already on the board
                    if roll !=6: # if token is not out, and 6 wasn't rolled, it is true that it cannot be moved.
                        state.append(True)
                        continue
                    else:
                        print(f'1')
                        return False
                else:
                    if roll + self.current_pos[token] > self.num_fields:
                        state.append(True)
                        continue
                    else:
                        print(f'2')
                        return False
                    
        else:
            for token in move.keys():
                if not self.tokens_out[token]: # if the token is not on the board
                    if token == selected and roll == 6 and move[token] == 1: # valid if token was moved and 6 was rolled
                        state.append(True)
                        continue
                    elif token != selected: # valid if token was not moved at all
                        state.append(True)
                        continue
                        
                    else:                   # False otherwise
                        print('f3')
                        return False
                    
                else:
                    if token == selected and self.current_pos[token] + roll == move[token]:
                        state.append(True)
                        continue
                    elif token != selected:
                        state.append(True)
                        continue
                    else:
                        print('f4')
                        return False
        
        if all(state):
            return True
                        
    

    def parse_reply(self, text):
        match = re.search(r"MY MOVE: X -> (\d+) ; Y -> (\d+)", text)
        if not match:
            raise ValueError("Invalid response format")
        
        x_value = int(match.group(1))
        y_value = int(match.group(2))
    

        return {'X' : x_value, 'Y': y_value}

    def update_board(self, move):
        split_b = self.board.split()
        
        for token in move.keys():
            pos = move[token]
            if self.tokens_out[token]:
                split_b[self.current_pos[token] - 1] = '□'
                split_b[pos - 1] = token
                
        new_board = ' '.join(split_b)
        self.current_state = new_board

    def add_message(self, message, role='user'):
        # add message to context
        if self.context == []:
            self.context = [
                {"role": "system", "content": self.sys_prompt}
            ]
        self.context.append({"role": role, "content": message})

    def reset(self):
        # resets game state
        self.turn = 0
        self.context = []


In [5]:
instance = Game(lmm, content, sys_prompt)


In [6]:
instance.make_move()

□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
X □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □


In [7]:
instance.last_response

'There are 23 empty fields and 0 occupied fields. In total: 23 + 0 = 23 fields. There are no tokens on the board. You have rolled a 6. This allows me to place my token X on the board at field 1.\n\nMY MOVE: X -> 1 ; Y -> 0'

In [8]:
instance.context


[{'role': 'system',
  'content': 'You are an avid board game player who likes to play the game according to the given rules.'},
 {'role': 'user',
  'content': 'Welcome to my new game! Your task is to move from start to end across the board. I will give you a board with empty fields. An empty field is marked like this □.\nYour tokens are marked with X and Y. Your tokens X or Y count as occupied fields. The total number of fields remains unchanged throughout the game.\n\n\nGeneral instructions:\n1) Every turn I will give you the current state of the grid. You must tell me what your next move is.\n2) Your answers must contain the words "MY MOVE:", the instruction of how you want the token to move.\n3) You must keep track of the state of the board in order to win the game.\n4) You cannot have 2 tokens in the same field.\n5) You can only play one token in 1 round.\n6) You can add a new token when 6 is rolled.\n7) If both tokens are on the board, no new tokens can be added.\n8) The number of

In [9]:
instance.check_move(move, roll)

NameError: name 'move' is not defined

In [ ]:
move = {'X': 1, 'Y': 0}
roll = 6

In [ ]:
def both_tokens_moved(self, move):
        l = []
        for token in move.keys():
            if self.current_pos[token] != move[token]:
                l.append(True)
            else:
                l.append(False)
        if all(l):
            return True
        else:
            return False

def find_selected(self, tokens_moved):
    selected = None
    for t in tokens_moved.keys():
        if tokens_moved[t] == True:
            selected = t
            
    return selected


def check_move():
    state = []
    both_tokens_moved = self.both_tokens_moved(move)
    
    if both_tokens_moved:
        return False
    
    
    tokens_moved = self.token_moved(move)
    selected = find_selected(tokens_moved)
    
    if not selected: # if no move has been made
        for token in move.keys(): # iterate through tokens 
            if not self.tokens_out[token]: # if token is not already on the board
                if roll !=6: # if token is not out, and 6 wasn't rolled, it is true that it cannot be moved.
                    state.append(True)
                    continue
                else:
                    return False
            else:
                if roll + self.current_pos[token] > self.num_fields:
                    state.append(True)
                    continue
                else:
                    return False
                
    else:
        for token in move.keys():
            
            if not self.tokens_out[token]: # if the token is not on the board
                if token == selected and roll == 6: # valid if token was moved and 6 was rolled
                    state.append(True)
                    continue
                elif token != selected: # valid if token was not moved at all
                    state.append(True)
                    continue
                    
                else:                   # False otherwise
                    return False
                
            else:
                if token == selected and self.current_pos + roll == move[token]:
                    state.append(True)
                    continue
                elif token != selected:
                    state.append(True)
                    continue
                else:
                    return False
                
                
                
                
    

In [ ]:
d = {'X' : 0, 'Y': 1}

d1 = {'X' : 0, 'Y': 2}




In [ ]:
    # def check_move(self, move, roll):
    #     state = []
    #     both_tokens_moved = self.both_tokens_moved(move)
    #     move_made = False
    #     
    #     if both_tokens_moved:
    #         print('f0')
    #         return False
    #     else:
    #         for token in move.keys():
    #             token_moved = self.token_moved(token, move)
    #             if not self.tokens_out[token]:
    #                 if roll == 6 and move[token] == 1: # we rolled a 6 and token is at position 1
    #                     if not move_made and token_moved: #if we still haven't moved a token this turn AND the current selected token is moved to 1
    #                         state.append(True)
    #                         move_made = True
    #                         continue
    #                         
    #                     else: #
    #                         print('f1')
    #                         return False
    #             
    #                 elif roll != 6 and move[token] == 0:
    #                     
    #                     state.append(True)
    #                     continue
    #                     
    #                 elif roll == 6 and move[token] == 0:
    #                     if move_made:
    #                         state.append(True)
    #                         continue
    #                     else:
    #                         print(f'failed here')
    #                         print(move_made)
    #                         return False
    #                  
    #                 else:
    #                     print(f'error 1 for token {token}')
    #                     return False
    #                 
    #             else:    
    #                 if roll + self.current_pos[token] == move[token]: 
    #                     if not move_made:
    #                         
    #                         state.append(True)
    #                         move_made = True
    #                         continue
    #                     if move_made:
    #                         print('f2')
    #                         return False
    #                 
    #                 elif self.current_pos[token] == move[token]:
    #                     if move_made:
    #                         state.append(True)
    #                         continue
    #                         
    #                     elif roll +  self.current_pos[token] > self.num_fields:
    #                         state.append(True)
    #                         continue
    #                     
    #                         
    #                     else:
    #                         print('f3')
    #                         return False
    #                 
    #                 elif move[token] > self.num_fields: 
    #                     print(f'error 2 for token {token}')
    #                     return False
    #                 
    #                 elif self.current_pos[token] > move[token]:
    #                     print(f'error 3 for token {token}')          
    #                     return False
    #                 
    #                 else:
    #                     print(f'error 4 for token {token}')                    
    #                     return False
    #                 
    #                 
    #     if all(state):
    #         return True
